<a href="https://colab.research.google.com/github/richdied/TensorflowDeveloperCertification/blob/main/Category3_German_Traffic_Signs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
####################################################
## [중요]
## 2023년 6월 1일 기준
## Google Colab에 설치된 텐서플로우(TensorFlow) 버전은 2.12 버전입니다.
## 시험에 제출 가능한 버전은 2.9.0 버전이기 때문에 버전을 다운그레이드 진행해야 합니다.
## 반드시 아래 코드를 실행하여 버전 다운그레이드 후 모델링을 진행하세요.
## 시험 볼 때는 현재 이 코드는 지워주셔야 합니다.
## 관련하여 궁금하신 점은 슬랙 커뮤니티에 질문 남겨 주세요.
####################################################
import urllib.request

url = 'https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/tfcert.py'
urllib.request.urlretrieve(url, 'tfcert.py')
%run tfcert.py

import tensorflow as tf
import tensorflow_datasets as tfds

print(f'설치 완료 후 TensorFlow 버전: {tf.__version__}')
print(f'설치 완료 후 TensorFlow Datasets 버전: {tfds.__version__}')

## 시험을 위한 버전 확인 ###########
## TensorFlow:          2.9.0 #
## TensorFlow Datasets: 4.6.0 #
###############################

설치환경: Google Colab
TensorFlow 시험환경을 구성중입니다. 잠시만 기다려 주세요.
(설치는 약 1~5분 정도 소요 됩니다)
[알림] TensorFlow 시험환경 구성이 완료 되었습니다.
설치 완료 후 TensorFlow 버전: 2.14.0
설치 완료 후 TensorFlow Datasets 버전: 4.6.0


In [3]:
# ================================================================
# There are 5 questions in this exam with increasing difficulty from 1-5.
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score significantly
# less than your Category 5 question.
#
# Don't use lambda layers in your model.
# You do not need them to solve the question.
# Lambda layers are not supported by the grading infrastructure.
#
# You must use the Submit and Test button to submit your model
# at least once in this category before you finally submit your exam,
# otherwise you will score zero for this category.
# ==============================================================================
#
# BASIC DATASETS QUESTION
#
# Create a classifier for the German Traffic Signs dataset that classifies
# images of traffic signs into 43 classes.
# ==============================================================================
#
# ABOUT THE DATASET
#
# The dataset contains traffic sign boards from the streets captured into
# image files. There are 43 unique classes in total. The images are of shape
# (30,30,3).
# ==============================================================================
#
# INSTRUCTIONS
#
# We have already divided the data for training and validation.
#
# Complete the code in following functions:
# 1. preprocess()
# 2. solution_model()
#
# Your code will fail to be graded if the following criteria are not met:
# 1. The input shape of your model must be (30,30,3), because the testing
#    infrastructure expects inputs according to this specification.
# 2. The last layer of your model must be a Dense layer with 43 neurons
#    activated by softmax since this dataset has 43 classes.
#
# HINT: Your neural network must have a validation accuracy of approximately
# 0.95 or above on the normalized validation dataset for top marks.

import urllib
import zipfile

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, Reshape, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint


# This function downloads and extracts the dataset to the directory that
# contains this file.
# DO NOT CHANGE THIS CODE
# (unless you need to change https to http)
def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/germantrafficsigns.zip'
    urllib.request.urlretrieve(url, 'germantrafficsigns.zip')
    with zipfile.ZipFile('germantrafficsigns.zip', 'r') as zip_ref:
        zip_ref.extractall()

# COMPLETE THE CODE IN THIS FUNCTION
def preprocess(image, label):
    # NORMALIZE YOUR IMAGES HERE (HINT: Rescale by 1/.255)
    image /= 255
    return image, label


# This function loads the data, normalizes and resizes the images, splits it into
# train and validation sets, defines the model, compiles it and finally
# trains the model. The trained model is returned from this function.

# COMPLETE THE CODE IN THIS FUNCTION.
def solution_model():
    # Downloads and extracts the dataset to the directory that
    # contains this file.
    download_and_extract_data()
    BATCH_SIZE = 32
    IMG_SIZE = 30

    # The following code reads the training and validation data from their
    # respective directories, resizes them into the specified image size
    # and splits them into batches. You must fill in the image_size
    # argument for both training and validation data.
    # HINT: Image size is a tuple
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory='train/',
        label_mode='categorical',
        image_size=(IMG_SIZE,IMG_SIZE),
        batch_size = BATCH_SIZE)

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory='validation/',
        label_mode='categorical',
        image_size=(IMG_SIZE,IMG_SIZE),
        batch_size = BATCH_SIZE)

    # Normalizes train and validation datasets using the
    # preprocess() function.
    # Also makes other calls, as evident from the code, to prepare them for
    # training.
    # Do not batch or resize the images in the dataset here since it's already
    # been done previously.

    train_ds = train_ds.map(
        preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(
        tf.data.experimental.AUTOTUNE)
    val_ds = val_ds.map(
        preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Code to define the model
    model = tf.keras.models.Sequential([
        Conv2D(32,(3,3), activation='relu', input_shape =(30,30,3)),
        MaxPooling2D(2,2),
        Conv2D(64,(3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(64,(3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dropout(0.25),
        Dense(128,activation='relu'),


        tf.keras.layers.Dense(43, activation=tf.nn.softmax)
    ])

    # Code to compile and train the model
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
    checkpoint_path = "tmp_checkpoint.ckpt"
    checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)


    model.fit(train_ds,
          validation_data=(val_ds),
          epochs=10,
          callbacks=[checkpoint],
          )
    model.load_weights(checkpoint_path)

    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")

Found 31367 files belonging to 43 classes.
Found 7842 files belonging to 43 classes.
Epoch 1/10
981/981 [==============================] - ETA: 0s - loss: 1.9074 - acc: 0.4511
Epoch 1: val_loss improved from inf to 0.65371, saving model to tmp_checkpoint.ckpt
981/981 [==============================] - 24s 12ms/step - loss: 1.9074 - acc: 0.4511 - val_loss: 0.6537 - val_acc: 0.8161
Epoch 2/10
978/981 [============================>.] - ETA: 0s - loss: 0.4846 - acc: 0.8489
Epoch 2: val_loss improved from 0.65371 to 0.23825, saving model to tmp_checkpoint.ckpt
981/981 [==============================] - 13s 13ms/step - loss: 0.4842 - acc: 0.8491 - val_loss: 0.2383 - val_acc: 0.9376
Epoch 3/10
975/981 [============================>.] - ETA: 0s - loss: 0.2631 - acc: 0.9179
Epoch 3: val_loss improved from 0.23825 to 0.12309, saving model to tmp_checkpoint.ckpt
981/981 [==============================] - 13s 13ms/step - loss: 0.2624 - acc: 0.9181 - val_loss: 0.1231 - val_acc: 0.9666
Epoch 4/10
97

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
